In [1]:
from pathlib import Path,PureWindowsPath 
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.losses import categorical_crossentropy
from keras.optimizers import adam, sgd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from PIL import Image

Using TensorFlow backend.


In [2]:
train_path=Path(r'C:\Users\Harshini\Documents\waste_segregation\DATASET\TRAIN')
test_path=Path(r'C:\Users\Harshini\Documents\waste_segregation\DATASET\TEST')
IMG_BREDTH = 30
IMG_HEIGHT = 60
num_classes = 2

In [3]:
train_batch = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=45, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=False).flow_from_directory(train_path, 
                                                                          target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                                          classes=['O','R','G','M','PA','PL','T','C'], 
                                                                          batch_size=100)

test_batch = ImageDataGenerator().flow_from_directory(test_path, 
                                                      target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                      classes=['O','R','G','M','PA','PL','T','C'], 
                                                      batch_size=100)

Found 24509 images belonging to 8 classes.
Found 3095 images belonging to 8 classes.


In [4]:
def cnn_model():    
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(IMG_HEIGHT,IMG_BREDTH,3)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
              
    model.summary()
              
    return model

def use_model(path):
    
    model = load_model(r'C:\Users\Harshini\Downloads\acdertya\best_waste_classifier.h5')
    pic = plt.imread(path)
    pic = cv2.resize(pic, (IMG_BREDTH, IMG_HEIGHT))
    pic = np.expand_dims(pic, axis=0)
    classes = model.predict_classes(pic)
    return classes

In [5]:
model = cnn_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 28, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 11, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 9, 64)        

In [7]:
checkpoint = ModelCheckpoint(r'C:\Users\Harshini\Downloads\acdertya\best_waste_classifier.h5', 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='auto')

In [8]:
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=1.0e-4), metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 28, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 11, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 9, 64)        

In [13]:
path=input("Enter the file path: ")
l=int(use_model(path))
#0-bio
#1-non-bio
#2-glass
#3-metal
#4-paper
#5-plastic
#6-trash
#7-cardboard

Enter the file path: C:\Users\Harshini\Documents\bio.jpg
0


In [26]:
from datetime import datetime 
from datetime import timedelta
if(l==0 or l==4 or l==7):
    str=datetime.now()+timedelta(days=2)
    type="bio"
else:
    str=datetime.now()+timedelta(days=1)
    type="non-bio"

In [36]:
import requests
import geocoder as rg 

# Step 1) Find the public IP of the user. This is easier said that done, look into the library Netifaces if you're
# interested in getting the public IP locally.
# The GeoIP API I'm going to use here is 'https://geojs.io/' but any service offering similar JSON data will work.

ip_request = requests.get('https://get.geojs.io/v1/ip.json')
my_ip = ip_request.json()['ip']  # ip_request.json() => {ip: 'XXX.XXX.XX.X'}
#print(my_ip)
# Prints The IP string, ex: 198.975.33.4

# Step 2) Look up the GeoIP information from a database for the user's ip

geo_request_url = 'https://get.geojs.io/v1/ip/geo/' + my_ip + '.json'
geo_request = requests.get(geo_request_url)
geo_data = geo_request.json()
print(geo_data['city']+" "+geo_data['country']+" "+geo_data['latitude']+" "+geo_data['longitude'])

Chennai India 13.086 80.2751


In [38]:
print(geo_data['city'],'Municipality:')
print('A complaint has been endorsed on the littering of ',type,' degradable waste. Hence this is to inform you to clear the waste by ',str);

Chennai Municipality:
A complaint has been endorsed on the littering of  bio  degradable waste. Hence this is to inform you to clear the waste by  2020-03-17 06:21:21.503717


In [39]:
print('Superindent Engineer:')
print('Cc')
print('A complaint has been endorsed on the littering of ',type,' degradable waste. Hence this is to inform you to clear the waste by ',str);

Superindent Engineer:
Cc
A complaint has been endorsed on the littering of  bio  degradable waste. Hence this is to inform you to clear the waste by  2020-03-17 06:21:21.503717
